In [1]:
import torch.optim as optim
import torch.nn as nn
from qiskit.providers.fake_provider import FakeLima
from scripts.data_setup import load_data
from scripts.from_circ_to_numpy import operations_to_features, save_to_json, load_from_json
from scripts.model import create_models, train_and_test_step, save_models, load_models

In [2]:
n_qubits = 5 # 5 == n_qubits, default value for now because of FakeLima
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/circuits/data_small_1')

In [3]:
X_train, y_train, X_test, y_test = load_from_json('data/features/data_small_1')

In [17]:
# specify these :)
sequence_input_size = 7
sequence_hidden_size = 8
sequence_num_layers = 4
sequence_model_type = 'LSTM'
sequence_dropout = 0
ann_hidden_layers = 2
ann_hidden_units = 64
ann_dropout = 0
noisy_first = True

sequence_model, ann = create_models(sequence_input_size, 
                                    sequence_hidden_size, 
                                    sequence_num_layers, 
                                    sequence_model_type, 
                                    sequence_dropout, 
                                    ann_hidden_layers, 
                                    ann_hidden_units, 
                                    ann_dropout, 
                                    noisy_first=noisy_first)

sequence_config = { 
    "input_size": sequence_input_size,
    "hidden_size": sequence_hidden_size,
    "num_layers": sequence_num_layers,
    "model_type": sequence_model_type,
    "dropout": sequence_dropout
}

ann_config = {
    "hidden_layers": ann_hidden_layers,
    "hidden_units": ann_hidden_units,
    "dropout": ann_dropout,
    "noisy_first": noisy_first
}

In [18]:
loss_fn = nn.MSELoss() # leave for now
optimizer = optim.Adam(list(ann.parameters()) + list(sequence_model.parameters()), lr=0.001)

num_epochs = 5
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, train loss (PyTorch): 0.2592, test_loss (scikitlearn rmse): 0.2208
Epoch 2/5, train loss (PyTorch): 0.0738, test_loss (scikitlearn rmse): 0.2001
Epoch 3/5, train loss (PyTorch): 0.0619, test_loss (scikitlearn rmse): 0.1957
Epoch 4/5, train loss (PyTorch): 0.0563, test_loss (scikitlearn rmse): 0.1935
Epoch 5/5, train loss (PyTorch): 0.0549, test_loss (scikitlearn rmse): 0.1932
